<a href="https://colab.research.google.com/github/fagonzalezo/qmc/blob/master/examples/TPU-QM-DM-mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install qmc if running in Google Colab

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
    !pip install --upgrade  git+https://github.com/fagonzalezo/qmc.git
else:
    import sys
    sys.path.insert(0, "../")

  Cloning https://github.com/fagonzalezo/qmc.git to /tmp/pip-req-build-vwbdkirz
  Running command git clone -q https://github.com/fagonzalezo/qmc.git /tmp/pip-req-build-vwbdkirz
  Created wheel for qmc: filename=qmc-0.0.1-cp37-none-any.whl size=11811 sha256=92bfe2aac56930e57e47df8e7ea8b28b942ee04a6efa9716b58684fc7ae5cc73
  Stored in directory: /tmp/pip-ephem-wheel-cache-2nu_8mvv/wheels/ff/c2/53/58cb50bb85949fb1dcda723a7c1f48a590515dee0aa40a4cc2
Successfully built qmc
  Found existing installation: qmc 0.0.1
    Uninstalling qmc-0.0.1:
      Successfully uninstalled qmc-0.0.1


In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import qmc.tf.layers as layers
import qmc.tf.models as models

In [3]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Running on TPU  ['10.31.33.242:8470']
INFO:tensorflow:Initializing the TPU system: grpc://10.31.33.242:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.31.33.242:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [4]:
BATCH_SIZE = 64 * tpu_strategy.num_replicas_in_sync
training_images_file   = 'gs://mnist-public/train-images-idx3-ubyte'
training_labels_file   = 'gs://mnist-public/train-labels-idx1-ubyte'
validation_images_file = 'gs://mnist-public/t10k-images-idx3-ubyte'
validation_labels_file = 'gs://mnist-public/t10k-labels-idx1-ubyte'
def read_label(tf_bytestring):
    label = tf.io.decode_raw(tf_bytestring, tf.uint8)
    label = tf.reshape(label, [])
    label = tf.one_hot(label, 10)
    return label
  
def read_image(tf_bytestring):
    image = tf.io.decode_raw(tf_bytestring, tf.uint8)
    image = tf.cast(image, tf.float32)/256.0
    image = tf.reshape(image, [28*28])
    return image
  
def load_dataset(image_file, label_file):
    imagedataset = tf.data.FixedLengthRecordDataset(image_file, 28*28, header_bytes=16)
    imagedataset = imagedataset.map(read_image, num_parallel_calls=16)
    labelsdataset = tf.data.FixedLengthRecordDataset(label_file, 1, header_bytes=8)
    labelsdataset = labelsdataset.map(read_label, num_parallel_calls=16)
    dataset = tf.data.Dataset.zip((imagedataset, labelsdataset))
    return dataset 
  
def get_training_dataset(image_file, label_file, batch_size):
    dataset = load_dataset(image_file, label_file)
    dataset = dataset.cache()  # this small dataset can be entirely cached in RAM
    dataset = dataset.shuffle(5000, reshuffle_each_iteration=True)
    dataset = dataset.repeat()
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(-1)  # fetch next batches while training on the current one (-1: autotune prefetch buffer size)
    return dataset
  
def get_validation_dataset(image_file, label_file):
    dataset = load_dataset(image_file, label_file)
    dataset = dataset.cache() # this small dataset can be entirely cached in RAM
    dataset = dataset.batch(10000)
    return dataset

# instantiate the datasets
training_dataset = get_training_dataset(training_images_file, training_labels_file, BATCH_SIZE)
validation_dataset = get_validation_dataset(validation_images_file, validation_labels_file)

In [22]:
with tpu_strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
    model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Softmax()
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(0.001),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=[tf.keras.metrics.CategoricalAccuracy()],
    )
EPOCHS = 10
steps_per_epoch = 60000//BATCH_SIZE  # 60,000 items in this dataset
print("Steps per epoch: ", steps_per_epoch)
  
history = model.fit(training_dataset,
                    steps_per_epoch=steps_per_epoch, epochs=EPOCHS,
                    validation_data=validation_dataset, validation_steps=1)

Steps per epoch:  117
Epoch 1/10


117/117 [==============================] - ETA: 0s - loss: 1.0396 - categorical_accuracy: 0.7108WARNING:tensorflow:Model was constructed with shape (None, 28, 28) for input KerasTensor(type_spec=TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='flatten_3_input'), name='flatten_3_input', description="created by layer 'flatten_3_input'"), but it was called on an input with incompatible shape (None, 784).


117/117 [==============================] - 5s 28ms/step - loss: 1.0358 - categorical_accuracy: 0.7119 - val_loss: 0.2807 - val_categorical_accuracy: 0.9227
Epoch 2/10
117/117 [==============================] - 3s 25ms/step - loss: 0.2689 - categorical_accuracy: 0.9259 - val_loss: 0.2147 - val_categorical_accuracy: 0.9390
Epoch 3/10
117/117 [==============================] - 2s 19ms/step - loss: 0.2063 - categorical_accuracy: 0.9418 - val_loss: 0.1821 - val_categorical_accuracy: 0.9480
Epoch 4/10
117/117 [==============================] - 2s 20ms/step - loss: 0.1695 - categorical_accuracy: 0.9515 - val_loss: 0.1580 - val_categorical_accuracy: 0.9552
Epoch 5/10
117/117 [==============================] - 2s 20ms/step - loss: 0.1455 - categorical_accuracy: 0.9589 - val_loss: 0.1390 - val_categorical_accuracy: 0.9606
Epoch 6/10
117/117 [==============================] - 2s 20ms/step - loss: 0.1253 - categorical_accuracy: 0.9646 - val_loss: 0.1263 - val_categorical_accuracy: 0.9631
Epoch 7/1

In [83]:
input_dim = 784
component_dim = 128
gamma = 2**-5
num_eig = 128
random_state = 0
LEARNING_RATE = 0.004
LEARNING_RATE_EXP_DECAY = 0.6 if tpu_strategy.num_replicas_in_sync == 1 else 0.7
lr_decay = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: LEARNING_RATE * LEARNING_RATE_EXP_DECAY**epoch,
    verbose=True)

with tpu_strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
    inputs = tf.keras.Input(shape=(784,))
    fm_x = layers.QFeatureMapRFF(784, dim=component_dim , gamma=gamma, random_state=random_state)
    psi_x = fm_x(inputs)
    ones = tf.ones_like(inputs[:, 0:1])
    rho_x = tf.keras.layers.concatenate((ones, psi_x), axis=1)
    rho_x = tf.expand_dims(rho_x, axis=-1)
    qmdmc = layers.QMeasureDMClassifEig(dim_x=component_dim , dim_y=10, eig_out=num_eig // 10, num_eig=num_eig)
    rho_y = qmdmc(rho_x)
    y_w = rho_y[:, 0, :] # shape (b, d_in)
    y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
    probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v))
    qmdmclf = tf.keras.Model(inputs=inputs, outputs=probs)
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001, decay=1e-6)
    qmdmclf.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])

EPOCHS = 10
steps_per_epoch = 60000//BATCH_SIZE  # 60,000 items in this dataset
print("Steps per epoch: ", steps_per_epoch)
  
history = qmdmclf.fit(training_dataset,
                    steps_per_epoch=steps_per_epoch, epochs=EPOCHS,
                    validation_data=validation_dataset, validation_steps=1,
                    callbacks=[lr_decay])

Steps per epoch:  117
Epoch 1/10

Epoch 00001: LearningRateScheduler reducing learning rate to 0.004.
117/117 [==============================] - 9s 39ms/step - loss: 1.7407 - accuracy: 0.4969 - val_loss: 0.9724 - val_accuracy: 0.8501
Epoch 2/10

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0028.
117/117 [==============================] - 3s 23ms/step - loss: 0.9490 - accuracy: 0.8586 - val_loss: 0.8807 - val_accuracy: 0.8838
Epoch 3/10

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00196.
117/117 [==============================] - 3s 23ms/step - loss: 0.8669 - accuracy: 0.8879 - val_loss: 0.8479 - val_accuracy: 0.8923
Epoch 4/10

Epoch 00004: LearningRateScheduler reducing learning rate to 0.0013719999999999997.
117/117 [==============================] - 3s 24ms/step - loss: 0.8317 - accuracy: 0.9033 - val_loss: 0.8308 - val_accuracy: 0.8884
Epoch 5/10

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0009603999999999998.
117/117 [======

In [95]:
input_dim = 784
component_dim = 128
gamma = 2**-5
num_eig = 128
random_state = 0
LEARNING_RATE = 0.001
LEARNING_RATE_EXP_DECAY = 0.6 if tpu_strategy.num_replicas_in_sync == 1 else 0.7
lr_decay = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: LEARNING_RATE * LEARNING_RATE_EXP_DECAY**epoch,
    verbose=True)

with tpu_strategy.scope(): 
    inputs = tf.keras.Input(shape=(784,))
    fm_x1 = layers.QFeatureMapRFF(784, dim=component_dim , gamma=gamma, random_state=random_state)
    psi_x = fm_x1(inputs)
    ones = tf.ones_like(inputs[:, 0:1])
    rho_x = tf.keras.layers.concatenate((ones, psi_x), axis=1)
    rho_x = tf.expand_dims(rho_x, axis=-1)
    qmdmc1 = layers.QMeasureDMClassifEig(dim_x=component_dim , dim_y=10, eig_out=num_eig // 10, num_eig=num_eig)
    rho_h = qmdmc1(rho_x)
    qmdmc2 = layers.QMeasureDMClassifEig(dim_x=10 , dim_y=10, eig_out=10, num_eig=10)
    rho_y = qmdmc2(rho_h)
    y_w = rho_y[:, 0, :] # shape (b, d_in)
    y_v = rho_y[:, 1:, :] # shape (b, dim_x, d_in)
    probs = tf.einsum('...j,...ij,...ij->...i', y_w, y_v, tf.math.conj(y_v))
    qmdmclf2 = tf.keras.Model(inputs=inputs, outputs=probs)
    opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)
    qmdmclf2.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=opt, metrics=['accuracy'])

fm_x1.set_weights(fm_x.get_weights())
fm_x1.trainable = False
qmdmc1.set_rho(qmdmc.get_rho())
qmdmc1.trainable = False
EPOCHS = 10
steps_per_epoch = 60000//BATCH_SIZE  # 60,000 items in this dataset
print("Steps per epoch: ", steps_per_epoch)
  
history = qmdmclf2.fit(training_dataset,
                    steps_per_epoch=steps_per_epoch, epochs=EPOCHS,
                    validation_data=validation_dataset, validation_steps=1, )
#                    callbacks=[lr_decay])

Steps per epoch:  117
Epoch 1/10
117/117 [==============================] - 14s 56ms/step - loss: 2.0220 - accuracy: 0.3398 - val_loss: 1.3194 - val_accuracy: 0.7304
Epoch 2/10
117/117 [==============================] - 3s 24ms/step - loss: 1.2232 - accuracy: 0.7712 - val_loss: 1.0075 - val_accuracy: 0.8085
Epoch 3/10
117/117 [==============================] - 3s 24ms/step - loss: 0.9885 - accuracy: 0.8221 - val_loss: 0.9440 - val_accuracy: 0.8106
Epoch 4/10
117/117 [==============================] - 3s 24ms/step - loss: 0.9250 - accuracy: 0.8383 - val_loss: 0.9601 - val_accuracy: 0.7430
Epoch 5/10
117/117 [==============================] - 3s 24ms/step - loss: 0.9071 - accuracy: 0.7880 - val_loss: 0.8822 - val_accuracy: 0.8436
Epoch 6/10
117/117 [==============================] - 3s 24ms/step - loss: 0.8912 - accuracy: 0.8501 - val_loss: 0.8582 - val_accuracy: 0.8443
Epoch 7/10
117/117 [==============================] - 3s 24ms/step - loss: 0.8494 - accuracy: 0.8508 - val_loss: 0.8213